In [33]:
import os
os.getcwd()

import sys 
sys.path.append(os.path.abspath(os.path.join('../')))
from langchain import LLMChain
from langchain import SagemakerEndpoint
from langchain.prompts import PromptTemplate
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import os
import json

from langchain.vectorstores import Pinecone

from langchain.chains import RetrievalQA

from langchain.chains.question_answering import load_qa_chain


from dotenv import load_dotenv
load_dotenv()




from config import *
from src.utils import connect_index

region = os.environ["AWS_REGION"]
endpoint_name = os.environ["SAGEMAKER_ENDPOINT_NAME"]
print(endpoint_name)

Falcon7b


In [34]:
embeddings = CohereEmbeddings(model=COHERE_MODEL_NAME, cohere_api_key=COHERE_API_KEY)

In [35]:
index = connect_index(PINECONE_INDEX_NAME)
vectorstore = Pinecone(index, embeddings.embed_query, "text")

2023-08-10 22:02:11.715 
  command:

    streamlit run /Users/isaac/FundamentlPartners/abinvenv-sol/venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-08-10 22:02:12.714 Connected to Pinecone index stab-test


In [5]:
parameters = {
    "do_sample": True,
    "top_p": 0.95,
    "temperature": 0.1,
    "max_new_tokens": 256,
    "num_return_sequences": 4,
}
    
class ContentHandler(LLMContentHandler):
        content_type = "application/json"
        accepts = "application/json"
        
        def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
            input_str = json.dumps({"inputs": prompt, "parameters": parameters, **model_kwargs})
            return input_str.encode('utf-8')
        
        def transform_output(self, output: bytes) -> str:
            response_json = json.loads(output.read().decode("utf-8"))
            return response_json[0]['generated_text']

In [38]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])



In [39]:
content_handler = ContentHandler()

llm=SagemakerEndpoint(
     endpoint_name=endpoint_name, 
     region_name=region, 
     model_kwargs={"max_new_tokens": 200, "top_p": 0.9, "max_length": None, "temperature":1e-10},
     endpoint_kwargs={"CustomAttributes": 'accept_eula=true'},
     content_handler=content_handler,
     credentials_profile_name="fundamentl-ai"
 )

2023-08-09 20:34:35.181 Found credentials in shared credentials file: ~/.aws/credentials


In [40]:
chain = load_qa_chain(prompt=QA_CHAIN_PROMPT, llm=llm)

In [41]:
question = "What is the capital of France?"

In [42]:
from langchain.docstore.document import Document
example_doc_1 = """
Peter and Elizabeth took a taxi to attend the night party in the city. While in the party, Elizabeth collapsed and was rushed to the hospital.
Since she was diagnosed with a brain injury, the doctor told Peter to stay besides her until she gets well.
Therefore, Peter stayed with her at the hospital for 3 days without leaving.
"""

docs = [
    Document(
        page_content=example_doc_1,
    )
]

In [43]:
#qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever(), chain_type="stuff", return_source_documents=True, chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

In [44]:
prompt_template = """
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it 
does not know.
{context}
Instruction: Based on the above documents, provide a detailed answer for, {question} 
Answer "I don't know" 
if info is not present in the document. 
Solution:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(), 
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

In [45]:
result = qa_chain({"query":"What is rainfall data?"})

In [46]:
result['result'].split('\n')[-1]

'Rainfall data is a type of data that is collected from weather stations or gauges. It is data that is collected on a regular basis and is used to understand the amount of precipitation that has occurred in a particular area. Rainfall data is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to understand the amount of precipitation that has occurred in a particular area. It is used to under

In [47]:
result['result'].split('\n')[6]

''

In [48]:
result['source_documents']

[Document(page_content='In order to help correct for the inaccuracies and errors that are present in the two types of data and take advantage of their individual strengths, research institutions have produced rainfall datasets that combine station and satellite/radar observations. Datasets that combine these observations are the recommended source of data for humanitarian purposes.\n\nObservational precipitation data can be presented in various formats, each providing unique insights into precipitation patterns. The most common representation is through absolute values, which reflect the actual amount of precipitation observed at a specific location and time, typically measured in millimeters. Precipitation averages, which are calculated from historical records over a period of typically at least 10 years, can be produced for a specific location, season, or year. A precipitation anomaly represents the deviation from the expected or average precipitation over a particular period, which 

In [49]:
result = qa_chain({"query": "What is Twitter?"})

In [52]:
result['result'].split('\n')[-1]

'In summary, Twitter is a social media platform that allows users to post and share short messages with other users. It is free to use and'

In [51]:
result['source_documents']

[Document(page_content="Enguehard, C. 1994. Acquisition of terminology from colloquial texts. In Proceedings, Computational Linguistics for Speech and Handwriting Recognition (CLSHR).\n\nGrefenstette, G. 1994. Explorations in Automatic Thesaurus Discovery. Dordrecht, The Netherlands: Kluwer Academic Publisher.\n\nJacquemin, C. 1994. Recycling terms into a partial parser. In Proceedings, 4th Conference on Applied Natural Language Processing (ANLP'94),  113-118.\n\nLewis, D. D., and Croft, W. B. 1990. Term clustering of syntactic phrasess. In Proceedings, 13th Annual International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR'90),  385-404.\n\nResnik, P. 1993. Selection and Information : A Class-Based Approach to Lexical Relationships. Ph.D. thesis, University of Pennsylvania, Institute for Research in Cognitive Science.\n\nShieber, S. N. 1986. An Introduction to Unification-Based Approaches to Grammar. CSLI Lecture Notes 4. Stanford, CA: CSLI.\n\nSmadj

In [23]:
from langchain.embeddings.sagemaker_endpoint import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
from typing import Dict, List, Optional

In [29]:
class ContentHandlerEmbeddings(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json


content_handler_embeddings = ContentHandlerEmbeddings()

In [30]:
stsb = SagemakerEndpointEmbeddings(endpoint_name="stsb", region_name="us-east-1", credentials_profile_name="fundamentl-ai", content_handler=content_handler_embeddings)

2023-08-10 21:59:04.541 INFO    botocore.credentials: Found credentials in shared credentials file: ~/.aws/credentials


In [41]:
query = "Que es la precipitacion?"

In [42]:
vectorstore_sagemaker = Pinecone(index, stsb.embed_query, "text")

In [43]:
vectorstore.similarity_search(query)

[Document(page_content='In order to help correct for the inaccuracies and errors that are present in the two types of data and take advantage of their individual strengths, research institutions have produced rainfall datasets that combine station and satellite/radar observations. Datasets that combine these observations are the recommended source of data for humanitarian purposes.\n\nObservational precipitation data can be presented in various formats, each providing unique insights into precipitation patterns. The most common representation is through absolute values, which reflect the actual amount of precipitation observed at a specific location and time, typically measured in millimeters. Precipitation averages, which are calculated from historical records over a period of typically at least 10 years, can be produced for a specific location, season, or year. A precipitation anomaly represents the deviation from the expected or average precipitation over a particular period, which 

In [32]:
stsb.embed_query(query)

[[[0.2647412121295929,
   -0.8787495493888855,
   1.4533323049545288,
   0.06026820093393326,
   0.8109136819839478,
   0.6753756403923035,
   -0.02625236287713051,
   0.9819003939628601,
   -0.37010854482650757,
   -0.3109547197818756,
   -0.08817920833826065,
   0.4888758659362793,
   -0.14285869896411896,
   0.5400943160057068,
   -0.34259477257728577,
   -0.017438380047678947,
   -0.19323429465293884,
   0.023980164900422096,
   0.3318628668785095,
   -0.4491732716560364,
   0.08117619901895523,
   1.5933806896209717,
   0.3798007071018219,
   0.25034916400909424,
   0.1705932766199112,
   -0.6115803718566895,
   -0.38859525322914124,
   -0.6368667483329773,
   0.941586971282959,
   0.0055649736896157265,
   -0.3151152431964874,
   0.21063095331192017,
   -0.7927321791648865,
   0.50898677110672,
   -0.07731356471776962,
   0.17164213955402374,
   -1.000885248184204,
   0.5939140319824219,
   -0.924860417842865,
   -0.24407880008220673,
   -0.8487597107887268,
   -1.060843467712402